In [2]:
import cv2
import os
import pickle
import mediapipe as mp
from tqdm import tqdm
from IPython.display import clear_output

2025-04-03 10:24:04.932136: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743643445.022717   22233 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743643445.049398   22233 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743643445.254616   22233 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743643445.254656   22233 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743643445.254657   22233 computation_placer.cc:177] computation placer alr

In [ ]:
# 데이터셋 경로
DATASET_DIR = '/home/shin/deeplearning-repo-1/dataset/front_lunge'

# 좌표 추출 운동 설정
exericse_name = "프론트 런지"

pickle_name = exericse_name + ".pkl"

exericse_type = ["스탠딩 니업", "숄더프레스", "스쿼트", "프론트 런지", "푸시업", "브릿지", "스탠딩 바이시클", "사이드 런지"]
exericse_label = exericse_type.index(exericse_name)

# 시퀀스 사이즈
SEQUENCE_LENGTH = 30

# mediapipe 선언
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

sequences = []
labels = []
landmark_points = [ 11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28 ]

with mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5) as pose:
    for video_file in tqdm(os.listdir(DATASET_DIR)):
        video_path = os.path.join(DATASET_DIR, video_file)
        cap = cv2.VideoCapture(video_path)
        frames = []
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(frame_rgb)
            if results.pose_landmarks:
                landmarks = []

                lm = results.pose_landmarks.landmark

                # 기준점
                neck_x = (lm[11].x + lm[12].x) / 2
                neck_y = (lm[11].y + lm[12].y) / 2
                #neck_z = (lm[11].z + lm[12].z) / 2

                for idx in landmark_points:
                    x = lm[idx].x - neck_x
                    y = lm[idx].y - neck_y
                    #z = lm[idx].z - neck_z
                    landmarks.append([x, y])
                frames.append(landmarks)
                if len(frames) >= SEQUENCE_LENGTH:
                    sequences.append(frames[-SEQUENCE_LENGTH:])
                    labels.append(exericse_label)
        cap.release()

I0000 00:00:1743656282.906553   22233 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1743656282.915057   98711 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.2.8-1ubuntu1~24.04.1), renderer: Mesa Intel(R) UHD Graphics (CML GT2)
I0000 00:00:1743656282.922902   22233 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1743656282.924429   98726 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.2.8-1ubuntu1~24.04.1), renderer: Mesa Intel(R) UHD Graphics (CML GT2)
  0%|          | 0/19 [00:00<?, ?it/s]

W0000 00:00:1743656283.009557   98698 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743656283.010282   98714 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743656283.048867   98722 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743656283.053683   98707 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
100%|██████████| 19/19 [02:45<00:00,  8.70s/it]


In [52]:
with open(pickle_name, 'wb') as f:
    pickle.dump((sequences, labels), f)

    print(exericse_name + " 데이터 저장 완료!")

프론트 런지 데이터 저장 완료!


In [ ]:
import os
import cv2
import numpy as np
import mediapipe as mp
from tqdm import tqdm
import pickle

# === 설정값
DATASET_DIR = '/home/shin/deeplearning-repo-1/dataset/front_lunge/front'
exericse_name = "프론트 런지"
pickle_name = exericse_name + ".pkl"
exericse_type = ["가만히", "스탠딩 니업", "숄더프레스", "스쿼트", "프론트 런지", "푸시업", "브릿지", "스탠딩 바이시클", "사이드 런지"]
exericse_label = exericse_type.index(exericse_name)
SEQUENCE_LENGTH = 20

# === 각도 계산 함수
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

# === MediaPipe 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

# === 결과 저장 리스트
sequences = []
labels = []

# === 관절 선택
landmark_points = [11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28]

# === 각도 조합 정의
angle_joints = [
    (11, 13, 15), (12, 14, 16),  # 양팔
    (23, 25, 27), (24, 26, 28),  # 양다리
    (13, 11, 23), (14, 12, 24),  # 상체
    (11, 23, 25), (12, 24, 26),  # 골반
]

# === 데이터 수집
for video_file in tqdm(os.listdir(DATASET_DIR)):
    video_path = os.path.join(DATASET_DIR, video_file)
    cap = cv2.VideoCapture(video_path)
    features_per_video = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(frame_rgb)
        if not results.pose_landmarks:
            continue

        lm = results.pose_landmarks.landmark
        neck_x = (lm[11].x + lm[12].x) / 2
        neck_y = (lm[11].y + lm[12].y) / 2
        landmarks = [(lm[i].x - neck_x, lm[i].y - neck_y) for i in range(33)]

        # x, y 좌표 (12개 관절만)
        xy_features = []
        for i in landmark_points:
            x, y = landmarks[i]
            xy_features.extend([x, y])

        # 관절 각도 (8개)
        angle_features = []
        for a, b, c in angle_joints:
            angle = calculate_angle(landmarks[a], landmarks[b], landmarks[c])
            angle_features.append(angle)

        full_feature = xy_features + angle_features
        features_per_video.append(full_feature)

    cap.release()

    # === 시퀀스 단위로 분할
    for i in range(len(features_per_video) - SEQUENCE_LENGTH):
        seq = features_per_video[i:i + SEQUENCE_LENGTH]
        sequences.append(seq)
        labels.append(exericse_label)

I0000 00:00:1743645527.970714   22233 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1743645527.972235   61437 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.2.8-1ubuntu1~24.04.1), renderer: Mesa Intel(R) UHD Graphics (CML GT2)
  0%|          | 0/10 [00:00<?, ?it/s]W0000 00:00:1743645528.037936   61427 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743645528.083486   61425 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
100%|██████████| 10/10 [02:04<00:00, 12.43s/it]


In [14]:
# # === 결과 저장
# output = {
#     "sequences": sequences,
#     "labels": labels
# }

with open(pickle_name, "wb") as f:
    pickle.dump((sequences, labels), f)

    print(exericse_name + " 데이터 저장 완료!")

프론트 런지 데이터 저장 완료!
